# Chapter 13 이진트리

## 13.2 이진 트리 구현하기

In [ ]:
def BinaryTreeList(r):
    return [r, [], []]

def insertLeft(root, newBranch):
    t = root.pop(1)
    if len(t) > 1:
        root.insert(1, [newBranch, t, []])
    else:
        root.insert(1, [newBranch, [], []])
    return root

def insertRight(root, newBranch):
    t = root.pop(2)
    if len(t) > 1:
        root.insert(2, [newBranch, [], t])
    else:
        root.insert(2, [newBranch, [], []])
    return root

def getRootVal(root):
    return root[0]

def setRootval(root, newVal):
    root[0] = newVal

def getLeftChild(root):
    return root[1]

def getRightChild(root):
    return root[2]

def main():
    r = BinaryTreeList(3)
    insertLeft(r, 4)
    insertLeft(r, 5)
    insertRight(r, 6)
    insertRight(r, 7)
    print(getRootVal(r))
    print(getLeftChild(r))
    print(getRightChild(r))

if __name__ == "__main__":
    main()


3
[5, [4, [], []], []]
[7, [], [6, [], []]]


In [1]:
"""
We want take this binary tree

                1           # Level 1
            2       3       # Level 2
        4       5           # Level 3
    6       7               # Level 4
8       9                   # Level 5


Property is next
    - Node size : n = 9
    - inner node number : b = n-1 = 8
    - Root node : 1
    - Max depth or height : h = 4
    - Is Balanced tree? No
    - Is binary search tree? No

"""



class Height(object):
    def __init__(self):
        self.height = 0

class NodeBT(object):
    def __init__(self, value=None, level=1):
        self.value = value
        self.level = level
        self.left = None
        self.right = None

    def __repr__(self):
        return f"{self.value}"

    def _add_next_node(self, value, level_here=2):
        new_node = NodeBT(value, level_here)
        if not self.value:
            self.value = new_node
        elif not self.left:
            self.left = new_node
        elif not self.right:
            self.right = new_node
        else:
            # 노드에 왼쪽 오른쪽 자식이 모두 있다면,
            # 왼쪽 자식 노드에 새 노드를 추가한다.
            # 그래서 예제의 트리가 왼쪽으로 치우쳐있다.
            self.left = self.left._add_next_node(value, level_here+1)
        return self

    def _search_for_node(self, value):
        # By the pre-order, take a value
        if self.value == value:
            return self
        else:
            found = None
            if self.left:
                found = self.left._search_for_node(value)
            if self.right:
                found = found or self.right._search_for_node(value)
            return found

    def _is_leaf(self):
        # 왼쪽, 오른쪽 자식이 모두 없는 노드
        return not self.right and not self.left

    def _get_max_height(self):
        # Take a maximum height - O(n)
        heightr, heightl = 0, 0
        if self.right:
            heightr = self.right._get_max_height() + 1
        if self.left:
            heightl = self.left._get_max_height() + 1
        return max(heightr,  heightl)

    def _is_balanced(self, height=Height()):
        # Is balanced tree? - O(n)
        lh = Height()
        rh = Height()

        if self.value is None:
            return True

        l, r = True, True
        if self.left:
            l = self.left._is_balanced(lh)
        if self.right:
            r = self.right._is_balanced(rh)

        height.height = max(lh.height, rh.height) + 1

        if abs(lh.height - rh.height) <= 1:
            return l and r

        return False

    def _is_bst(self, left=None, right=None):
        # Is binary searched tree? - O(n)
        if self.value:
            if left and self.value < left:
                return False
            if right and self.value > right:
                return False

            l, r = True, True
            if self.left:
                l = self.left._is_bst(left, self.value)
            if self.right:
                r = self.right._is_bst(self.value, right)
            return  l and r
        else:
            return True

class BinaryTree(object):
    def __init__(self):
        self.root = None

    def add_node(self, value):
        if not self.root:
            self.root = NodeBT(value)
        else:
            self.root._add_next_node(value)

    def is_leaf(self, value):
        node = self.root._search_for_node(value)
        if node:
            return node._is_leaf()
        else:
            return False

    def get_node_level(self, value):
        node = self.root._search_for_node(value)
        if node:
            return node.level
        else:
            False

    def is_root(self, value):
        return self.roo.value == value

    def get_height(self):
        return self.root._get_max_height()

    def is_balanced(self):
        return self.root._is_balanced()
    
    def is_bst(self):
        return self.root._is_bst()


if __name__ == "__main__":
    bt = BinaryTree()
    for i in range(1, 10):
        bt.add_node(i)
    print("8 is leaf?", bt.is_leaf(8))


8 is leaf? True


## 13.3 이진 탐색 트리
* 노드의 왼쪽 하위 트리에는 노드의 값보다 작은 값의 노드만 존재한다.
* 노드의 오른쪽 하위 트리에는 노드의 값보다 큰 값의 노드만 존재한다.
* 왼쪽과 오른쪽 하위 트리 모두 이진 탐색 트리여야함.
* 중복 노드가 없어야함.
* 균형 트리면 검색/삽입/삭제 시간복잡도 O(logn)

### 13.3.1 이진 탐색 트리 구현하기
앞의 이진 트리 클래스를 슈퍼클래스로 삼아 트리 구현

In [4]:
"""
다음 그림의 이진 탐색 트리를 구현한다.
                            6               ---> 레벨 1
                    4               8       ---> 레벨 2
                2       5       7       9   ---> 레벨 3
            1       3                       ---> 레벨 4

속성은 다음과 같다.
    - 노드의 개수(크기) : n = 9
    - 분기(or 내부 노드) 수 : b = n-1 = 8
    - 루트 노드 : 6
    - 최대 깊이 or 높이 : h = 3
    - 균형 트리입니까? YES
    - 이진 탐색 트리입니까? YES
"""

# from binary_tree import NodeBT, BinaryTree

class NodeBST(NodeBT):

    def __init__(self, value=None, level=1):
        self.value = value
        self.level = level
        self.left = None
        self.right = None

    def _add_next_node(self, value, level_here=2):
        new_node = NodeBST(value, level_here)
        if value > self.value:
            self.right = self.right and self.right._add_next_node(
                value, level_here + 1) or new_node
        elif value < self.value:
            self.left = self.left and self.left._add_next_node(
                value, level_here + 1) or new_node
        else:
            print("중복 노드를 허용하지 않습니다.")
        return self

    def _search_for_node(self, value):
        if self.value == value:
            return self
        elif self.left and value < self.value:
            return self.left._search_for_node(value)
        elif self.right and value > self.value:
            return self.right._search_for_node(value)
        else:
            return False

class BinarySearchTree(BinaryTree):
    def __init__(self):
        self.root = None

    def add_node(self, value):
        if not self.root:
            self.root = NodeBST(value)
        else:
            self.root._add_next_node(value)

if __name__ == "__main__":
    bst = BinarySearchTree()
    # for i in range(10):
    for i in [6, 4, 8, 2, 5, 7, 9, 1, 3]:
        bst.add_node(i)
    print("8은 말단 노드?", bst.is_leaf(8))
    print("트리의 높이는?", bst.get_height())

8은 말단 노드? False
트리의 높이는? 3


## 13.4 자가 균형 이진 탐색 트리
* 균형트리 : 모든 하위 트리의 높이 차가 1 이하인 트리
* 자가 균형 이진 탐색 트리 : 노드의 삽입과 삭제 같은 연산이 일어날 때 자동으로 균형 트리를 유지하는 이진 탐색 트리
* 균형도 : 왼쪽과 오른쪽 하위 트리 높이의 차이
* 노드 분할 및 병합
* 노드 회전

13.4.1 AVL 트리
* 왼쪽과 오른쪽 하위 트리의 높이 차이가 1보다 클 수 없는 자체 균형 조건을 가진 이진 탐색 트리.

### 13.4.2 레드-블랙 트리
* 노드는 레드 or 블랙
* 루트 노드는 블랙
* 모든 널 포인터(NIL)는 블랙
* 레드 노드의 양쪽 자식 노드는 언제나 블랙
* 어떤 노드부터 말단 노드에 도달하는 모든 경로에는, 말단 노드를 제외하면 모두 같은 개수의 블랙 노드가 있음.

### 13.4.3 이진 힙
완전 균형 이진 트리임. 이진 힙의 유일한 작업은 부모 노드와 자식 노드를 교체 하는 것.